# BWA Alignment
This is a notebook walk through for Burrows-Wheerler Alignment (BWA) for Whole Genome Variant analysis. 

## FASTQ Quality Check
This is the first step for quality checking your samples. To check your FASTQ files, use FASTQC which is published by the [Babraham Bioinformatics](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) team as the first line QC tool.

This tool will evaluate the overall health of your sample. In particular, you will want to look at:
- Total number of sequences
    - Make sure it aligns with your targetted expectations
- Per base sequencing quality
    - It should drop over time, especially in 2x300bp reads. Ensure that it doesnt drop too far, usually you want Q30>90% if possible
- QC content
    - Make sure it aligns with your genome of interest
- %N
    - N is an ambiguous base. You dont want many of these in your sample (if any)
- Sequence Length
    - Make sure the length you put in to the sequencer is whats coming out
- Duplication levels
    - Keep as low as possible, but any library with PCR you expect some
- Adapter content
    - This will be driven by your insert size and sequencing length. If you are sequencing through your entire insert and in to the adapter on the other side, you would expect contamination here. Usually best to avoid that, but if you see some you will need to trim it out. 

In [ ]:
%%bash
fastqc raw_align/SAMN09914146_1.fastq.gz
fastqc raw_align/SAMN09914146_2.fastq.gz

To view your FASTQC output, navigate into the raw_align directory (containing the raw files for alignment) and look for .html files that are the output. Download those and then open them in your browser to explore the output.

## Read Trimming
Trimming the reads is often a critical step in preparing your samples for alignment. This remove ambiguous bases, low quality bases/reads, and also removes any adapter content that may be in the read. For most trimming cases, [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic) is a viable tool. Another commonly used tool is [CutAdapt](https://cutadapt.readthedocs.io/en/stable/), but here we will use trimmomatic. 

Parameters used:
- PE
    - Denotes the input as paired end reads. Following that, you need to put the two input files, then trimmed-1-unpaired, trimmed-1-paired, trimmed-2-unpaired, trimmed-2-paired for output files
- ILLUMINACLIP
    - To clip off specifically any Illumina reads that are found. The numbers that follow are the seed mismatch (maximum count of mismatches to identify adapter), palindrome clip threshold (to remove possible identical adapters on both ends of reads), and the simple clip threshold (how accurate the adapter is to read beyond the seed)
- LEADING TRAILING
    - Removes bases from the start and end of the read if they are below thresholds. "2" is Illumina for "low quality"
- SLIDING WINDOW
    - Looks at the read in a sliding window frame and takes the average of multiple bases. The first number is the window size and the second is quality score. Again "2" is Illumina for "low quality", but this allows for 1 base to not potentially trash an otherwise high quality read. 
- MINLEN
    - The minimum length of a read after trimming to accept. 

In [ ]:
%%bash
trimmomatic PE raw_align/SAMN09914146_1.fastq.gz raw_align/SAMN09914146_2.fastq.gz \
    raw_align/trimmed_R1_unpaired.fastq.gz raw_align/trimmed_R1_paired.fastq.gz\
    raw_align/trimmed_R2_unpaired.fastq.gz raw_align/trimmed_R2_paired.fastq.gz\
    ILLUMINACLIP:raw_align/adapters.fasta:2:40:15 \
    LEADING:2 TRAILING:2 \
    SLIDINGWINDOW:4:2 \
    MINLEN:25

And then do a QC check on the samples again to see how they are after trimming!

We also introduce a tool called [MultiQC](https://multiqc.info/). This is a great QC aggregation tool. Play around with it a bit, but after running this you can download the multiqc file, open it in a browser, and view things aggregated. 

In [ ]:
%%bash
fastqc raw_align/trimmed_R1_unpaired.fastq.gz 
fastqc raw_align/trimmed_R1_paired.fastq.gz
fastqc raw_align/trimmed_R2_unpaired.fastq.gz 
fastqc raw_align/trimmed_R2_paired.fastq.gz
multiqc raw_align/.

## BWA
Now we are getting to the actual alignment steps using the [Burrows-Wheeler Aligner](http://bio-bwa.sourceforge.net/bwa.shtml). If you remember back to the lecture, BWA is based upon a compressed suffix array for alignment. Therefore, to do the alignment you must first index your reference genome. This converts it to the compressed suffix array format that will allow it to be appropriately scanned. 

In [ ]:
%%bash
bwa index raw_align/NZ_CP017669.1.fa

Once its been indexed, you can now align your reads to the reference. Specifically within BWA, there are three base algorithms. MEM is the most recent and most accurate, so rarely will you see a use case to not use bwa mem. 

Primary Parameters:
- Indexed Reference
    - The first parameter after mem is your Indexed Reference file. It should be the same as the file you indexed the step previously
- The various FASTQ files you will be aligning. 
- Output file
    - The is after the carrot (>). The default for BWA is the sam file format.
If all you need is a quick standard alignment, that is all you need. 

Advanced Paramters:
For tweaking the algorithm operation. Think back to the smith-waterman and BLAST alignment discussions. A lot of these parameters might sound familiar...
- -k
    - Minimum seed length. Default 19
- -w 
    - Band width. Gaps longer than it will not be found. Default 100
- -d
    - Z dropoff. Avoids unnecessary extension and reduces poor alignments in good alignments. Default 100
- -r
    - Re-seed trigger. Larger number means less seeds, which is faster but less accurate. Default 1.5
- -c
    - Discard if it has more than X occurences in the genome. To remove overly prolific reads (rare). Default 10000
- -A
    - Matching score. Default 1
- -B
    - Mismatch penalty. Default 4
- -O
    - Open gap penalty. Default 6
- -E
    - Gap extension penalty. Default 1
- -L 
    - Clipping penalty. If the final alignment score is > the best score-this, it wont be clipped. Default 5
- -U
    - Unpaired read penalty. Default 9
And a few others that are less likely to be used. If you are interested more, explore the BWA documentation. 


In [ ]:
%%bash
bwa mem raw_align/NZ_CP017669.1.fa raw_align/trimmed_R1_paired.fastq.gz raw_align/trimmed_R2_paired.fastq.gz > ecoli.sam

## Samtools management
Now we will use [Samtools](http://www.htslib.org/) to manipulate the data into a more useable output. In this case we will first convert it to a .bam file (allows for indexing), then we will sort the file, then index it to allow for easy and quick callbacks later. 

In [ ]:
%%bash
samtools view -hSbo ecoli.bam ecoli.sam
samtools sort ecoli.bam -o ecoli.sorted.bam
samtools index ecoli.sorted.bam

We can also use Samtools to look at some stats now. One option is the flagstat command, but you can also just use stat. Both tell you different information, so try them both out!

In [ ]:
%%bash
samtools flagstat ecoli.sorted.bam

The next step is to perform the variant calling. A standard tool for this is within samtools called [mpileup](http://www.htslib.org/doc/samtools-mpileup.html) and then [bcftools](http://samtools.github.io/bcftools/bcftools.html) to generate a Variant Call File (.vcf). 
Parameters: 
- -u
    - Sets output as VCF file
- -f
    - input reference
- -mv
    - Default calling method and variants only
- -Ov
    - Output format for VCF
Which we then follow up by downloading the *E.coli* reference annotations and intersecting with our variant list. We now can see where the variants are in terms of gene locations!

In [ ]:
%%bash
samtools mpileup -u -f raw_align/NZ_CP017669.1.fa ecoli.sorted.bam | \
    bcftools call -mv -Ov > variants.vcf
wget -O ecoli_genes.gff "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?db=nuccore&report=gff3&id=NZ_CP017669.1"
bedtools intersect -a ecoli_genes.gff -b variants.vcf -wa -u > aligned_variants.vcf